In [1]:
# Import the libraries
import os, json, ast, re
import openai
import pandas as pd

!pip install -qU openai
import openai
from google.colab import userdata
openai.api_key = userdata.get('OpenAI-Personal-Key')

# Read the Laptop Dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
filepath = '/content/drive/MyDrive/GenAI_Course_Master/Course_1_ShopAssistAI/'

from tenacity import retry, wait_random_exponential, stop_after_attempt
#### declare all functions in this section : BOC
def dialogue_mgmt_system():

    conversation = initialize_conversation()

    introduction = get_chat_model_completions(conversation)
    print(introduction.content)
    top_3_laptops = None

    user_input = input("")
    while(user_input != "exit"):
        moderation = chat_with_auto_moderation(user_input)
        if moderation == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break
        conversation.append({"role": "user", "content": user_input})
        # Step 1: Pass the user input to GPT
        response_assistant = get_chat_model_completions(conversation)
        moderation = chat_with_auto_moderation(response_assistant)
        if moderation == 'Flagged':
            print("Sorry, this message has been flagged. Please restart your conversation.")
            break
        if response_assistant.function_call:
            # Step 2: Extract top3 laptops by calling the external function
            function_name = response_assistant.function_call.name
            function_args = json.loads(response_assistant.function_call.arguments)

            if function_name == "compare_laptops_with_user":
                print("\nThank you for providing all the information. Kindly wait, while I fetch the products\n")
                top_3_laptops = compare_laptops_with_user(**function_args)
                function_response = recommendation_validation(top_3_laptops)
                if len(function_response) == 0:
                  print("Sorry, we do not have laptops that match your requirements. Connecting you to a human expert.")
                  break
                # Step 3: send the info on the function call and function response to GPT
                conversation.append(response_assistant)
                conversation.append(
                    {
                       "role": "function",
                       "name": function_name,
                       "content": function_response,
                    }
                )
                recommendation = get_chat_model_completions(conversation)
                conversation.append({"role": "assistant", "content": recommendation.content})
                print("\n" +recommendation.content + "\n")
        else:
           conversation.append({"role": "assistant", "content": response_assistant.content})
           print("\n" +  response_assistant.content + "\n")
        user_input = input("")

## Function Description for the Function Calling API
function_descriptions = [
            {
                "name": "compare_laptops_with_user",
                "description": "Get the top 3 laptops from the catalogue, that best matches what the user is asking based on 'GPU intensity','Display quality','Portability','Multitasking','Processing speed' & 'Budget",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "gpu intensity": {"type": "string","description": "The requirement of the user in GPU capacity classfied as low, medium or high"},
                        "display quality": {"type": "string", "description": "The requirement of the user for Laptop's Display Quality & capacity classfied as low, medium or high"},
                        "portability": {"type": "string", "description": "The requirement of the user for Laptop's portability classfied as low, medium or high"},
                        "multitasking": {"type": "string", "description": "The requirement of the user for Laptop's Multitasking classfied as low, medium or high"},
                        "processing speed": {"type": "string", "description": "The requirement of the user for Laptop's Processing speed classfied as low, medium or high"},
                        "budget": {"type": "integer", "description": "The maximum budget of the user"}
                    },
                    "required": ["GPU intensity","Display quality","Portability","Multitasking","Processing speed","Budget"]
                }
            },
            {
               "name": "moderate_content",
               "description": "Check if the content violates moderation policies.",
               "parameters": {
                  "type": "object",
                  "properties": {
                      "text": {"type": "string", "description": "The text to be checked for moderation issues."}
                  },
                  "required": ["text"]
                }
            }
    ]

# Main chat function
def chat_with_auto_moderation(user_input):
    # Step 1: Send user input to GPT with function calling enabled
    # Convert user_input to string if it's not already
    if not isinstance(user_input, str):
        user_input = user_input.content  # Access the 'content' attribute if it's an OpenAIObject

    # Step 2: Send user input to GPT with function calling enabled
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": user_input}],
        functions=function_descriptions,
        function_call="auto"  # Allow LLM to decide when to call moderation
    )

    # Step 3: Extract the message object
    response_message = response.choices[0].message
    #print("response message is  :" + str(response_message))


    # Step 4: Check if function_call is present
    if response_message.function_call:
        function_name = response_message.function_call.name
        function_args = json.loads(response_message.function_call.arguments) if response_message.function_call.arguments else {}

        if function_name == "moderate_content":
            # Step 4: Call the moderation function with extracted arguments
            #print("entered moderate content block")
            moderation_result = moderate_content(**function_args)
            return str(moderation_result)

    # Step 5: If no moderation was triggered, return the generated response
    #print("entered alternate block skipping moderation block")
    return response_message.content  if response_message.content else "No response generated."


# Define a function called moderation_check that takes user_input as a parameter.
def moderate_content(text):
    """
    Perform moderation check on user input using the OpenAI API.

    Parameters:
    - user_input (str): The text input to be checked for moderation.

    Returns:
    - str: "Flagged" if the input is flagged by the moderation system,
           "Not Flagged" otherwise.
    """
    if len(text) < 5:  # Avoid unnecessary API calls for trivial inputs
        return "Not Flagged"

    response = openai.moderations.create(input=text)
     # Extract the moderation result from the API response.
    moderation_output = response.results[0].flagged
    # Check if the input was flagged by the moderation system.
    if response.results[0].flagged == True:
        # If flagged, return "Flagged"
        return "Flagged"
    else:
        # If not flagged, return "Not Flagged"
        return "Not Flagged"""



def extract_dictionary_from_string(string):
    regex_pattern = r"\{[^{}]+\}"

    dictionary_matches = re.findall(regex_pattern, string)

    # Extract the first dictionary match and convert it to lowercase
    if dictionary_matches:
        dictionary_string = dictionary_matches[0]
        dictionary_string = dictionary_string.lower()

        # Convert the dictionary string to a dictionary object using ast.literal_eval()
        dictionary = ast.literal_eval(dictionary_string)

    return dictionary

def recommendation_validation(laptop_recommendation):
    data = json.loads(laptop_recommendation)
    data1 = []
    for i in range(len(data)):
        if data[i]['Score'] > 2:
            data1.append(data[i])

    return data1


def compare_laptops_with_user(user_req_string):
    ##from google.colab import drive
    ##drive.mount('/content/drive',force_remount=True)
    ##filepath = '/content/drive/MyDrive/GenAI_Course_Master/Course_1_ShopAssistAI/'
    ##df = pd.read_csv(filepath + 'laptop_descriptions.csv')
    ##laptop_df= pd.read_csv(filepath + 'laptop_data.csv')
    laptop_df = pd.read_csv(filepath +'updated_laptop.csv')
    user_requirements = extract_dictionary_from_string(user_req_string)
    budget = int(user_requirements.get('budget', '0').replace(',', '').split()[0])
    #This line retrieves the value associated with the key 'budget' from the user_requirements dictionary.
    #If the key is not found, the default value '0' is used.
    #The value is then processed to remove commas, split it into a list of strings, and take the first element of the list.
    #Finally, the resulting value is converted to an integer and assigned to the variable budget.

    filtered_laptops = laptop_df.copy()
    filtered_laptops['Price'] = filtered_laptops['Price'].str.replace(',','').astype(int)
    filtered_laptops = filtered_laptops[filtered_laptops['Price'] <= budget].copy()
    #These lines create a copy of the laptop_df DataFrame and assign it to filtered_laptops.
    #They then modify the 'Price' column in filtered_laptops by removing commas and converting the values to integers.
    #Finally, they filter filtered_laptops to include only rows where the 'Price' is less than or equal to the budget.

    mappings = {
        'low': 0,
        'medium': 1,
        'high': 2
    }
    # Create 'Score' column in the DataFrame and initialize to 0
    filtered_laptops['Score'] = 0
    for index, row in filtered_laptops.iterrows():
        user_product_match_str = row['laptop_feature']
        laptop_values = extract_dictionary_from_string(user_product_match_str)
        score = 0

        for key, user_value in user_requirements.items():
            if key.lower() == 'budget':
                continue  # Skip budget comparison
            laptop_value = laptop_values.get(key, None)
            laptop_mapping = mappings.get(laptop_value.lower(), -1)
            user_mapping = mappings.get(user_value.lower(), -1)
            if laptop_mapping >= user_mapping:
                ### If the laptop value is greater than or equal to the user value the score is incremented by 1
                score += 1

        filtered_laptops.loc[index, 'Score'] = score


    # Sort the laptops by score in descending order and return the top 3 products
    top_laptops = filtered_laptops.drop('laptop_feature', axis=1)
    top_laptops = top_laptops.sort_values('Score', ascending=False).head(3)

    return top_laptops.to_json(orient='records')


# Define a Chat Completions API call
# Retry up to 6 times with exponential backoff (same as original)
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def get_chat_model_completions(messages, json_format=False):
    MODEL = "gpt-3.5-turbo"

    # If JSON output is required, use function calling
    if json_format:
        chat_completion_json = openai.chat.completions.create(
            model=MODEL,
            messages=messages,
            functions=function_descriptions,
            function_call="auto",  # Let GPT decide when to call the function
            seed=1234
        )

        message = chat_completion_json.choices[0].message

        if "function_call" in message:
            function_args = json.loads(message.function_call.arguments)
            return function_args  # Output in {"output": "JSON content"} format

    # Otherwise, return plain text response
    chat_completion = openai.chat.completions.create(
        model=MODEL,
        messages=messages,
        seed=2345
    )

    return chat_completion.choices[0].message


def initialize_conversation():
    '''
    Returns a list [{"role": "system", "content": system_message}]
    '''
    delimiter = "#####"

    sample_user_req_dictionary = {'GPU intensity': "high",
                        'Display quality':"high",
                        'Portability': "medium",
                        'Multitasking': "low",
                        'Processing speed': "low",
                        'Budget': "125000"}

    sample_user_req_template = {'GPU intensity': "_",
                        'Display quality':"_",
                        'Portability': "_",
                        'Multitasking': "_",
                        'Processing speed': "_",
                        'Budget': "_"}


    system_message = f"""
    You are an intelligent laptop gadget expert and your goal is to find the best laptop for a user.
    You are trying to understand the user's requirement for these laptop's features: ('gpu intensity','display quality','portability','multitasking','processing speed','budget')
    You need to ask relevant questions and understand the user need for each feature by analysing the user's responses.
    After understanding their requirements, you'll use a function call to suggest the top 3 laptops with their respective user match score.
    Recommend these laptops and answer any user's query about them.

    {delimiter} Here are certain guidelines that you need to follow:
    Don't ask questions about more than 2 features at a time.
    If the user's budget is less than says less than 25000 INR, please mention that there are no laptops in that range.
    Recommend the top3 laptops in the following format:
    Start with a brief summary of each laptop in the following format, in decreasing order of price of laptops:
    1. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    2. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    3. <Laptop Name> : <Major specifications of the laptop>, <Price in Rs>
    {delimiter}

    {delimiter}To find the top3 laptops, you need to have the following chain of thoughts:
    Thought 1: Ask one question to understand the user's profile and requirements. \n
    If their primary use for the laptop is unclear. Ask another question to comprehend their needs.
    Answer "Yes" or "No" to indicate if you understand the requirements. \n
    If yes, proceed to the next step. Otherwise, rephrase the question to capture their profile. \n

    Thought 2: Now, you are trying to understand the requirements for other features which you couldn't in the previous step.
    Ask questions to strengthen your understanding of the user's profile.
    Don't ask questions about more than 2 features at a time.
    Answer "Yes" or "No" to indicate if you understood all the needs of the features and are confident about the same.
    If yes, move to the next Thought. If no, ask question on the features whose needs you are unsure of. \n
    It is a good practice to ask question with a sound logic as opposed to directly citing the feature you want to understand the need for.{delimiter}
    {delimiter}

    {delimiter} Here is a sample conversation between the user and assistant:
    Assistant: "Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?"
    User: "Hi, I am an editor."
    Assistant: "Great! As an editor, you likely require a laptop that can handle demanding tasks. Hence, the laptop should have high multitasking capability. You would also need a high end display for better visuals and editing. May I know what kind of work do you primarily focus on? Are you more involved in video editing, photo editing, or both? Understanding the specific type of editing work will help me tailor my recommendations accordingly. Let me know if my understanding is correct until now."
    User: "I primarily work with After Effects."
    Assistant: "Thank you for providing that information. Working with After Effects involves working with graphics, animations, and rendering, which will require high GPU. Do you work with high-resolution media files, such as 4K videos or RAW photos? Understanding your file sizes will help determine the storage capacity and processing power needed."
    User: "Yes, sometimes I work with 4K videos as well."
    Assistant: "Thank you for the information. Processing 4K vidoes will require a good processor and high GPU. I think we have already determined earlier that you need a high GPU. To ensure I have a complete understanding of your needs, I have one more question: Are you frequently on the go and require a laptop that is lightweight and easy to carry, or do you primarily work from a stationary location?"
    User: "Yes, sometimes I travel but do not carry my laptop."
    Assistant:"Could you kindly let me know your budget for the laptop? This will help me find options that fit within your price range while meeting the specified requirements."
    User: "my max budget is 1.5lakh inr"
    {delimiter}


    Start with a short welcome message and encourage the user to share their requirements.
    """
    conversation = [{"role": "system", "content": system_message}]

    return conversation

#### declare all functions in this section : EOC

#Calling the dialogue_management_system triggers the laptop recommendation chatbot
dialogue_mgmt_system()



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.5/474.5 kB 7.0 MB/s eta 0:00:00
Mounted at /content/drive
Hello! I'm here to help you find the perfect laptop that suits your needs. Could you please share your requirements?
Hi, I need a laptop for ML and AI development. This requires a high processing machine with High GPU. My budget is maximum 55000 INR.

Great! As an ML and AI developer, you need a laptop with high processing power and a strong GPU. To ensure efficient development and training of models, you require a laptop that can handle intensive computing tasks. Let me clarify one point: Do you prioritize having a high-end display quality for better visualization of data and models on your laptop screen?

Yes

Perfect! So, you are looking for a laptop with high processing power, a strong GPU for ML and AI development, and a high-quality display for better visualization. This combination of features will enhance your programming and debugging experience. To confirm, are you also i

[link text](https:// [link text](https://))